In [1]:

import Formatting

include("jlFiles/printmat.jl")
include("jlFiles/OlsFn.jl")
include("jlFiles/HszDkFn.jl")
include("jlFiles/HDirProdFn.jl")

HDirProdFn (generic function with 1 method)

In [2]:
ER1 = readdlm("Data/PPM_ER1.csv",',')                   #load data from csv files
ER2 = readdlm("Data/PPM_ER2.csv",',')
ER  = [ER1;ER2]
#ER = randn(2354,2637)           #uncomment this line (and comment the previous 3 lines)
                                 #if you do not have ER1.csv and ER2.csv

Factors   = readdlm("Data/PPM_Factors.csv",',')         #no header line: x is matrix
Investors = readdlm("Data/PPM_N_Changes.csv",',')
N_Changes = Investors[:,1]

(T,N) = size(ER)
D = N_Changes .> 50                      #logical dummies: [active]

2637-element BitArray{1}:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true

In [3]:
alphaM = fill(NaN,N)                                #individual alphas
for i = 1:N
   b, = OlsFn(ER[:,i],[Factors ones(T,1)])
   alphaM[i] = b[end]
end

println("\nAverage annualised alphas for the two groups")
printmat([mean(alphaM[~D]) mean(alphaM[D])]*252)


Average annualised alphas for the two groups
    -0.787     6.217



In [4]:
println("\nLS, group by group")

PortfER      = fill(NaN,(T,2))     #create portfolios as average across individuals
PortfER[:,1] = mean(ER[:,~D],2)   #Tx1, portfolio return = average individual return
PortfER[:,2] = mean(ER[:,D],2)

Avg = mean(PortfER,1)*252          #average excess return on portfolios
Std = std(PortfER,1)*sqrt(252)
SR  = Avg./Std
(b,res,yhat,Covb) = OlsFn(PortfER,[ones(T,1) Factors])

println("Stats for the portfolios:\n Avg, Std, SR, alpha")
printmat([Avg' Std' SR' b[1:1,:]'*252])

R       = [1 0 0 0 -1 0 0 0]                       #testing if alpha(1) = alpha(2)
a_diff  = R*vec(b)                                 
tstatLS = a_diff/sqrt(R*Covb*R')

println("diff of annual alpha (inactive - 51+), tstat (LS)")
printmat([a_diff*252 tstatLS])


LS, group by group
Stats for the portfolios: Avg, Std, SR, alpha
    -1.262    15.728    -0.080    -0.787
     5.534    13.882     0.399     6.217

diff of annual alpha (inactive - 51+), tstat (LS)
    -7.004    -2.780



In [5]:
println("\npanel")
(theta,CovDK,CovLS) = HszDkFn(ER,[ones(T,1) Factors],[~D D]+0.0)

R       = [1 0 0 0 -1 0 0 0]                #testing if alpha(1) = alpha(2)
a_diff  = R*theta
tstatLS = a_diff/sqrt(R*CovLS*R')
tstatDK = a_diff/sqrt(R*CovDK*R')

println("diff of annual alpha (inactive - 51+), tstat (LS), tstat (DK)")
printmat([a_diff*252 tstatLS tstatDK])


panel
diff of annual alpha (inactive - 51+), tstat (LS), tstat (DK)
    -7.004   -24.017    -2.784

